In [2]:
import os
import time
import glob
import re
from functools import reduce
import numpy as np
import uproot
import uproot_methods
import awkward
import pandas as pd
from klepto.archives import dir_archive


import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
#this cell for plotting NN score
import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

from memory_profiler import profile

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [4]:
def pad_and_flatten(val): 
    try:
        return val.pad(1, clip=True).fillna(0.).flatten()#.reshape(-1, 1)
    except AttributeError:
        return val.flatten()

#model = tf.keras.models.load_model('../ML/data/training.h5')#, custom_objects=None, compile=False)

#model._make_predict_function()
#graph = tf.get_default_graph()

#def run_model(inputs):
#    global graph
#    with graph.as_default():
#        outputs = model.predict(inputs)
#    return outputs

os.environ['KERAS_BACKEND'] = 'theano'
from keras.models import load_model

#model = load_model('../ML/data/training.h5')

Using Theano backend.


In [5]:
import sys
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [6]:
#Let's define our processor first. 

class WHhadProcessor(processor.ProcessorABC):
    def __init__(self):
        
        ## load the NN
        self.model = load_model('../ML/data/lostLep_Z_backgrounds/training.h5')
        self.stds  = pd.read_json('../ML/data/lostLep_Z_backgrounds/stds.json').squeeze()
        self.means = pd.read_json('../ML/data/lostLep_Z_backgrounds/means.json').squeeze()
        
        #Great, now let's define some bins for our histograms.
        
        dataset_axis         = hist.Cat("dataset", "Primary dataset")
        pt_axis              = hist.Bin("pt", r"$p_{T}$ (GeV)", 500, 0, 2000)
        #pt_axis              = hist.Bin("pt", r"$p_{T}$ (GeV)", 15, 0, 300)
        multiplicity_axis    = hist.Bin("multiplicity", r"N", 30, -0.5, 29.5)
        phi_axis             = hist.Bin("phi", r"$\Delta \phi$", 80, 0, 8)
        mass_axis            = hist.Bin("mass", r"mass (GeV)", 500, 0, 2000)
        r_axis               = hist.Bin("r", r"$\Delta R$", 80, 0, 4)
        score_axis           = hist.Bin("score", r"NN Score", 10, 0, 1)

        #In order to create proper histograms, we always need to include a dataset axis!
        #For different types of histograms with different scales, I create axis to fit 
        #those dimensions!
        
        #Now, let's move to actually telling our processor what histograms we want to make.
        #Let's start out simple. 
        self._accumulator = processor.dict_accumulator({
            "h_pt_met200_CR":                       hist.Hist("Counts", dataset_axis, pt_axis),
            "h_pt_met400_CR":                       hist.Hist("Counts", dataset_axis, pt_axis),
            "h_pt_met600_CR":                       hist.Hist("Counts", dataset_axis, pt_axis),
            
            "fj_pt_met200_CR":                      hist.Hist("Counts", dataset_axis, pt_axis),
            "fj_pt_met400_CR":                      hist.Hist("Counts", dataset_axis, pt_axis),
            "fj_pt_met600_CR":                      hist.Hist("Counts", dataset_axis, pt_axis),

            "h_mass_met200_CR":                     hist.Hist("Counts", dataset_axis, mass_axis),
            "h_mass_met400_CR":                     hist.Hist("Counts", dataset_axis, mass_axis),
            "h_mass_met600_CR":                     hist.Hist("Counts", dataset_axis, mass_axis),
            
            "met_noHiggs":                          hist.Hist("Counts", dataset_axis, pt_axis),
            "met_oneHiggs":                         hist.Hist("Counts", dataset_axis, pt_axis),
            "met_oneW":                             hist.Hist("Counts", dataset_axis, pt_axis),
            "met_inHiggs":                          hist.Hist("Counts", dataset_axis, pt_axis),
            "met_outHiggs":                         hist.Hist("Counts", dataset_axis, pt_axis),
            
            "met_CR":                               hist.Hist("Counts", dataset_axis, pt_axis),
            "met_SR":                               hist.Hist("Counts", dataset_axis, pt_axis),
            "met_Higgs_CR":                         hist.Hist("Counts", dataset_axis, pt_axis),
            "met_Higgs_SR":                         hist.Hist("Counts", dataset_axis, pt_axis),
            "met_W_CR":                             hist.Hist("Counts", dataset_axis, pt_axis),
            "met_W_SR":                             hist.Hist("Counts", dataset_axis, pt_axis),
            "met_Higgs_W_CR":                       hist.Hist("Counts", dataset_axis, pt_axis),
            "met_Higgs_W_SR":                       hist.Hist("Counts", dataset_axis, pt_axis),

        })

    
    @property
    
    def accumulator(self):
        return self._accumulator

    def process(self, df):
     
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ## MET -> can switch to puppi MET
        met_pt  = df["MET_pt"]
        met_phi = df["MET_phi"]
        
        ## Muons
        muon = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt = df['Muon_pt'].content,
            eta = df['Muon_eta'].content,
            phi = df['Muon_phi'].content,
            mass = df['Muon_mass'].content,
            miniPFRelIso_all=df['Muon_miniPFRelIso_all'].content,
            looseId =df['Muon_looseId'].content
            )
        muon = muon[(muon.pt > 10) & (abs(muon.eta) < 2.4) & (muon.looseId) & (muon.miniPFRelIso_all < 0.2)]
        #muon = Collections(df, "Muon", "tightTTH").get() # this needs a fix for DASK
        
        electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'].content, 
            eta=df['Electron_eta'].content, 
            phi=df['Electron_phi'].content,
            mass=df['Electron_mass'].content,
            pdgid=df['Electron_pdgId'].content,
            mini_iso=df['Electron_miniPFRelIso_all'].content
        )
        
        ## Electrons
        electron = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt = df['Electron_pt'].content,
            eta = df['Electron_eta'].content,
            phi = df['Electron_phi'].content,
            mass = df['Electron_mass'].content,
            miniPFRelIso_all=df['Electron_miniPFRelIso_all'].content,
            cutBased=df['Electron_cutBased'].content
            )
        electron = electron[(electron.pt>10) & (abs(electron.eta) < 2.4) & (electron.miniPFRelIso_all < 0.1) &  (electron.cutBased >= 1)]
        #electron = Collections(df, "Electron", "tightTTH").get() # this needs a fix for DASK
        
        tau = JaggedCandidateArray.candidatesfromcounts(
            df['nTau'],
            pt=df['Tau_pt'].content, 
            eta=df['Tau_eta'].content, 
            phi=df['Tau_phi'].content,
            mass=df['Tau_mass'].content,
            decaymode=df['Tau_idDecayMode'].content,
            newid=df['Tau_idMVAnewDM2017v2'].content,
        )
        tau = tau[(tau.pt > 20) & (abs(tau.eta) < 2.4) & (tau.decaymode) & (tau.newid >= 8)]
        
        isotrack = awkward.JaggedArray.zip(
            pt=df['IsoTrack_pt'], 
            eta=df['IsoTrack_eta'], 
            phi=df['IsoTrack_phi'], 
            rel_iso=df['IsoTrack_pfRelIso03_all'], 
        )
        isotrack = isotrack[(isotrack.pt > 10) & (abs(isotrack.eta) < 2.4) & ((isotrack.rel_iso < 0.1) | ((isotrack.rel_iso*isotrack.pt) < 6))]
        
        ## FatJets
        fatjet = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt = df['FatJet_pt'].content,
            eta = df['FatJet_eta'].content,
            phi = df['FatJet_phi'].content,
            mass = df['FatJet_mass'].content,
            msoftdrop = df["FatJet_msoftdrop"].content,  
            deepTagMD_HbbvsQCD = df['FatJet_deepTagMD_HbbvsQCD'].content, 
            deepTagMD_WvsQCD = df['FatJet_deepTagMD_WvsQCD'].content, 
            deepTag_WvsQCD = df['FatJet_deepTag_WvsQCD'].content
            
        )
        
        leadFatJet = fatjet[:,:1]
        leadingFatJets = fatjet[:,:2]
        difatjet = leadingFatJets.choose(2)
        dphiDiFatJet = np.arccos(np.cos(difatjet.i0.phi-difatjet.i1.phi))
        
        htag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        htag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        
        lead_htag = htag[htag.pt.argmax()]
        
        wtag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        wtag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        
        lead_wtag = wtag[wtag.pt.argmax()]
        
        wh = lead_htag.cross(lead_wtag)
        wh_deltaPhi = np.arccos(wh.i0.phi - wh.i1.phi)
        wh_deltaR = wh.i0.p4.delta_r(wh.i1.p4)
        
        ## Jets
        jet = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt = df['Jet_pt'].content,
            eta = df['Jet_eta'].content,
            phi = df['Jet_phi'].content,
            mass = df['Jet_mass'].content,
            jetId = df['Jet_jetId'].content, # https://twiki.cern.ch/twiki/bin/view/CMS/JetID
            #puId = df['Jet_puId'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/PileupJetID
            btagDeepB = df['Jet_btagDeepB'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/BtagRecommendation102X
            #deepJet = df['Jet_'].content # not there yet?
        )
        
        jet       = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        btag      = jet[(jet.btagDeepB>0.4184)]
        light     = jet[(jet.btagDeepB<0.4184)]
        
        ## Get the leading b-jets
        high_score_btag = jet[jet.btagDeepB.argsort(ascending=False)][:,:2]
        
        leading_jet    = jet[jet.pt.argmax()]
        leading_b      = btag[btag.pt.argmax()]
        
        bb = high_score_btag.choose(2)
        bb_deltaPhi = np.arccos(np.cos(bb.i0.phi-bb.i1.phi))
        bb_deltaR = bb.i0.p4.delta_r(bb.i1.p4)
        
        mtb = mt(btag.pt, btag.phi, met_pt, met_phi)
        min_mtb = mtb.min()
        
        ## other variables
        ht = jet.pt.sum()
        
        min_dphiJetMet4 = np.arccos(np.cos(jet[:,:4].phi-met_phi)).min()
        
        leadingJets = jet[:,:2]
        dijet = leadingJets.choose(2)
        dphiDiJet = np.arccos(np.cos(dijet.i0.phi-dijet.i1.phi))

        ## evaluate NN
        # first, prepare the inputs.
        # A .max() can ensure that the flattened array has the full length, but we rather use our pad_and_flatten function        
        # sorting in training: ['mll', 'njet', 'nbtag', 'st', 'ht', 'met', 'mjj_max', 'mlb_min', 'mlb_max', 'l0_pt', 'l1_pt', 'deltaR_lj_min', 'j0_pt']
        
        '''NN_inputs = np.stack([
            # normalize
            pad_and_flatten( (metpt - self.means['met'])/self.stds['met'] ),
            pad_and_flatten( (ht - self.means['ht'])/self.stds['ht'] ),
            pad_and_flatten( (lead_jet_pt - self.means['lead_jet_pt'])/self.stds['lead_jet_pt'] ),
            pad_and_flatten( (sublead_jet_pt - self.means['sublead_jet_pt'])/self.stds['sublead_jet_pt'] ),
            pad_and_flatten( (njets - self.means['njets'])/self.stds['njets'] ),
            pad_and_flatten( (nbjets - self.means['bjets'])/self.stds['bjets'] ),
            pad_and_flatten( (wtagged_mc.counts - self.means['nWs'])/self.stds['nWs'] ),
            pad_and_flatten( (htagged.counts - self.means['nHs'])/self.stds['nHs'] ),
            pad_and_flatten( (nfatjets - self.means['nFatJets'])/self.stds['nFatJets'] ),
            pad_and_flatten( (met_sig - self.means['met_significance'])/self.stds['met_significance'] ),
            pad_and_flatten( (abs_min_dphi_met_leadjs4 - self.means['min_dphi_met_j4'])/self.stds['min_dphi_met_j4'] ),
        ])
        
        NN_inputs = np.moveaxis(NN_inputs, 0, 1)
        NN_score = self.model.predict(NN_inputs)'''
        
        #filters
        good_vertices = df["Flag_goodVertices"]
        tighthalo = df["Flag_globalSuperTightHalo2016Filter"]
        noise_filter = df["Flag_HBHENoiseFilter"]
        noise_isofilter = df["Flag_HBHENoiseIsoFilter"]
        ecal_deadcell = df["Flag_EcalDeadCellTriggerPrimitiveFilter"]
        bad_pfmuon = df["Flag_BadPFMuonFilter"]
        ee_badsc = df["Flag_eeBadScFilter"]
       
        #trigger
        hlt_pfmet_250 = df["HLT_PFMET250_HBHECleaned"]
        hlt_pfmet_300 = df["HLT_PFMET300_HBHECleaned"]
        hlt_pfmet1_200 = df["HLT_PFMETTypeOne200_HBHE_BeamHaloCleaned"]
        hlt_pfmet_mht = df["HLT_PFMET120_PFMHT120_IDTight_PFHT60"]
        hlt_pfmetNoMu_mhtNoMu = df["HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60"]
        
       
        #Now it's time to make some selections.

        ht_ps = (ht > 300)
        met_g250 = (met_pt>250)
        met_l400 = (met_pt<400)
        met_bin1 = met_g250 & met_l400
        met_g400 = (met_pt>400)
        met_l600 = (met_pt<600)
        met_bin2 = met_g400 & met_l600
        met_bin3 = (met_pt>600)
        njet_cut = (jet.counts>=2)
        njet_veto = (jet.counts<=5)
        njet_ps = njet_cut & njet_veto
        bjet_ps = (btag.counts>=1)
        fatjet_sel = (fatjet.counts >=1)
        inc_fatjet_sel = (fatjet.counts >=2)
        #mt_sel = (min_mt_b_met > 200).any()
        
        min_dphi_sel = (min_dphiJetMet4>0.5)
        dphi_sel = (dphiDiJet.min()<2.5)
        fatjet_dphi_sel = (dphiDiFatJet<2.5).all()

        lowerHiggs = ((lead_htag.msoftdrop>90).any())
        upperHiggs = ((lead_htag.msoftdrop<150).any())
        
        lowerHiggsInv = ((lead_htag.msoftdrop<90).any())
        upperHiggsInv = ((lead_htag.msoftdrop>150).any())
        
        higgsWindow = lowerHiggs & upperHiggs
        invertedHiggsWindow = lowerHiggsInv | upperHiggsInv
        
        e_sel = (electron.counts == 0)
        m_sel = (muon.counts == 0)
        it_sel = (isotrack.counts == 0)
        t_sel = (tau.counts == 0)
        #l_sel = e_sel & m_sel & it_sel & t_sel
        l_sel = ((electron.counts + muon.counts) == 1)
        l_veto = (e_sel & m_sel & t_sel & it_sel)
        
        h_sel =(htag.counts>0) 
        wmc_sel = (wtag.counts>0) 

        met_fsel = (good_vertices == 1) & (tighthalo == 1) & (noise_filter == 1) & (noise_isofilter == 1) & (ecal_deadcell == 1) & (bad_pfmuon == 1) & (ee_badsc == 1) 
        met_tsel = (hlt_pfmet_250 == 1).any() | (hlt_pfmet_300 == 1).any() | (hlt_pfmet1_200 == 1).any() | (hlt_pfmet_mht == 1).any() | (hlt_pfmetNoMu_mhtNoMu == 1).any()
        
        #sel = ht_ps & met_ps & njet_ps & bjet_ps & l_sel & h_sel & wmc_sel
        #sel = ht_ps & met_ps & njet_ps & bjet_ps & fatjet_sel & l_sel & h_sel & min_dphi_sel & dphi_sel & fatjet_dphi_sel
        sel1_CR = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & h_sel & wmc_sel & met_bin1
        sel2_CR = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & h_sel & wmc_sel & met_bin2
        sel3_CR = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & h_sel & wmc_sel & met_bin3
        
        #selCR = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & h_sel & wmc_sel & met_g250 #& higgsWindow
        selCRNoHCut = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 #& h_sel & wmc_sel 
        selCROneHCut = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & h_sel #& wmc_sel 
        selCROneWCut = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & wmc_sel #& h_sel 
        selCRInverted = met_fsel & met_tsel & l_sel & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & h_sel & wmc_sel & met_g250 & invertedHiggsWindow 
    
        selCR       = met_fsel & met_tsel & l_sel  & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250
        selSR       = met_fsel & met_tsel & l_veto & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250
        selHiggsCR  = met_fsel & met_tsel & l_sel  & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & h_sel
        selHiggsSR  = met_fsel & met_tsel & l_veto & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & h_sel
        selWCR      = met_fsel & met_tsel & l_sel  & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & wmc_sel
        selWSR      = met_fsel & met_tsel & l_veto & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & wmc_sel
        selHiggsWCR = met_fsel & met_tsel & l_sel  & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & h_sel & wmc_sel
        selHiggsWSR = met_fsel & met_tsel & l_veto & njet_ps & bjet_ps & min_dphi_sel & dphi_sel & fatjet_dphi_sel & ht_ps & inc_fatjet_sel & met_g250 & h_sel & wmc_sel
        
        #Let's make sure we weight our events properly.
        #wght = df['weight'][sel] * 137
        weight = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']
        lumi = 1 if dataset=='Data' else 60

        wght1_CR = weight[sel1_CR] * lumi
        wght2_CR = weight[sel2_CR] * lumi
        wght3_CR = weight[sel3_CR] * lumi
        
        wghtCR      = weight[selCR] * lumi
        wghtSR      = weight[selSR] * lumi
        wghtHiggsCR = weight[selHiggsCR] * lumi
        wghtHiggsSR = weight[selHiggsSR] * lumi
        wghtWCR     = weight[selWCR] * lumi
        wghtWSR     = weight[selWSR] * lumi
        wghtHiggsWCR     = weight[selHiggsWCR] * lumi
        wghtHiggsWSR     = weight[selHiggsWSR] * lumi
        
        dataNormNoHCut = 1 if dataset=='Data' else 0.812
        wghtCRNoHCut = weight[selCRNoHCut] * lumi * dataNormNoHCut
        dataNormOneHCut = 1 if dataset=='Data' else 0.708
        wghtCROneHCut = weight[selCROneHCut] * lumi * dataNormOneHCut
        dataNormOneWCut = 1 if dataset=='Data' else 0.579
        wghtCROneWCut = weight[selCROneWCut] * lumi * dataNormOneWCut
        wghtCRInv = weight[selCRInverted] * lumi
        
        
        #Let's fill some histograms. 
        output['h_pt_met200_CR'].fill(dataset=dataset, pt=lead_htag[sel1_CR].pt.flatten(), weight=wght1_CR)
        output['h_pt_met400_CR'].fill(dataset=dataset, pt=lead_htag[sel2_CR].pt.flatten(), weight=wght2_CR)
        output['h_pt_met600_CR'].fill(dataset=dataset, pt=lead_htag[sel3_CR].pt.flatten(), weight=wght3_CR)

        output['fj_pt_met200_CR'].fill(dataset=dataset, pt=leadFatJet[sel1_CR].pt.flatten(), weight=wght1_CR)
        output['fj_pt_met400_CR'].fill(dataset=dataset, pt=leadFatJet[sel2_CR].pt.flatten(), weight=wght2_CR)
        output['fj_pt_met600_CR'].fill(dataset=dataset, pt=leadFatJet[sel3_CR].pt.flatten(), weight=wght3_CR)

        output['h_mass_met200_CR'].fill(dataset=dataset, mass=lead_htag[sel1_CR].mass.flatten(), weight=wght1_CR)
        output['h_mass_met400_CR'].fill(dataset=dataset, mass=lead_htag[sel2_CR].mass.flatten(), weight=wght2_CR)
        output['h_mass_met600_CR'].fill(dataset=dataset, mass=lead_htag[sel3_CR].mass.flatten(), weight=wght3_CR)

        output['met_noHiggs'].fill(dataset=dataset, pt=met_pt[selCRNoHCut].flatten(), weight=wghtCRNoHCut)
        output['met_oneHiggs'].fill(dataset=dataset, pt=met_pt[selCROneHCut].flatten(), weight=wghtCROneHCut)
        output['met_oneW'].fill(dataset=dataset, pt=met_pt[selCROneWCut].flatten(), weight=wghtCROneWCut)
        output['met_inHiggs'].fill(dataset=dataset, pt=met_pt[selCR].flatten(), weight=wghtCR)
        output['met_outHiggs'].fill(dataset=dataset, pt=met_pt[selCRInverted].flatten(), weight=wghtCRInv)
        
        output['met_CR'].fill(dataset=dataset, pt=met_pt[selCR].flatten(), weight=wghtCR)
        output['met_SR'].fill(dataset=dataset, pt=met_pt[selSR].flatten(), weight=wghtSR)
        output['met_Higgs_CR'].fill(dataset=dataset, pt=met_pt[selHiggsCR].flatten(), weight=wghtHiggsCR)
        output['met_Higgs_SR'].fill(dataset=dataset, pt=met_pt[selHiggsSR].flatten(), weight=wghtHiggsSR)
        output['met_W_CR'].fill(dataset=dataset, pt=met_pt[selWCR].flatten(), weight=wghtWCR)
        output['met_W_SR'].fill(dataset=dataset, pt=met_pt[selWSR].flatten(), weight=wghtWSR)
        output['met_Higgs_W_CR'].fill(dataset=dataset, pt=met_pt[selHiggsWCR].flatten(), weight=wghtHiggsWCR)
        output['met_Higgs_W_SR'].fill(dataset=dataset, pt=met_pt[selHiggsWSR].flatten(), weight=wghtHiggsWSR)
        
        return output

    
    def postprocess(self, accumulator):
        return accumulator

In [7]:
tag = '0p1p27'

fileset_WH  = {'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WH_had_750_1_nanoAOD/*.root'),
                'WJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/W*JetsToLNu_Tune*/*.root'),
                #'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/QCD_HT*/*.root'),
                'TTJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/TTJets*/*.root'),
                #'ZNuNu': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZJetsToNuNu*/*.root'),
                'ST': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST*/*.root'),
                #'ST_tW': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST_tW*/*.root'),
                #'ST_tChannel': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST_t-channel*/*.root'),
                #'ST_sChannel': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST_s-channel*/*.root'),
                'ttW/ttZ': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttWJets*/*.root')
                    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttZJets*/*.root'),
                #'WW/WZ/ZZ': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WW*/*.root')
                #    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WZ*/*.root')
                #    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2L2Nu*/*.root')
                #    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2Q2Nu*/*.root')
                }

fileset_WH_merge = {'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WH_had_750_1_nanoAOD/*.root'),
                'ZNuNu': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZJetsToNuNu*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttZJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WZ*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2L2Nu*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2Q2Nu*/*.root'),
                'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/QCD_HT*/*.root'),
                'LL': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/W*JetsToLNu_Tune*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/TTJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttWJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WW*/*.root'),
                #'Data': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p24/MET_Run2018*/*.root')

                }

#Here, I've separated by data from my background. This lets me change the style of the
#signal line and keep the background consistent. 

output = processor.run_uproot_job(fileset_WH,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 12, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )

#Here, we have the ability to change the 'workers' and 'chunksize', but to be honest,
#it does not make that much of a difference unless you want to see your progress bar 
#get updates more or less often. Totally a person choice.

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


HBox(children=(HTML(value='Preprocessing'), FloatProgress(value=0.0, max=223.0), HTML(value='')))

HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=223.0), HTML(value='')))

In [8]:
#Here's a block to help you make some pretty histos. This deals solely with style.
#These first two variables define my signal line as red and defines the error bar style I want!
lineopts = {
    'color': 'r'}

data_err_opts = {
    'linestyle': 'none',
    'marker': '_',
    'markersize': 10.,
    'color': 'r',
    'elinewidth': 1}

data_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1}

#Now, let's go to background. I define the line and fill color for the background because
#I want it to be a different style from my signal. 

lineopts2 = {
    'color': [('#8EA604'), ('#F5BB00') ,('#466365')],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
}
fillopts = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#1982C4')],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillopts1 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#8AC926'), ('#FFCA3A') ],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

'''def savefig(hists, outdir, name):
    import re
    bkgandsig = re.compile('(?!Data)')
    bkganddata = re.compile('(?!mC750_l1)')
    ax = hist.plot1d(hists[bkgandsig][bkganddata], overlay="dataset", density=False, stack=True, 
                fill_opts = fillopts1, overflow = 'over')
    hist.plot1d(hists['mC750_l1'], overlay="dataset", density=False, stack=False, 
                error_opts=data_err_opts, overflow = 'over') 
    hist.plot1d(hists['Data'], overlay="dataset", density=False, stack=False, 
                error_opts=data_err_opts_rat, overflow = 'over') 
    ax.set_yscale('log')
    ax.set_ylim(0.001,1000000)
    ax.figure.savefig(os.path.join(outdir, "{}_log.pdf".format(name)))
    ax.clear()

def savefigshape(hists, outdir, name):
    ax = hist.plot1d(hists, overlay="dataset", density=True, stack=False, 
                line_opts = lineopts2, overflow = 'over')
    
    ax.set_yscale('log')
    ax.set_ylim(0.00001,10)
    ax.figure.savefig(os.path.join(outdir, "{}_shape_log.pdf".format(name)))
    ax.clear()'''
    
def savefig(hists, outdir, name):
    import re
    bkgandsig = re.compile('(?!Data)')
    bkganddata = re.compile('(?!mC750_l1)')
    allbutLL = re.compile('(?!LL)')
    
    background = hists[bkgandsig][bkganddata]
    otherbackgrounds = hists[bkgandsig][bkganddata][allbutLL]
    lostLep = hists['LL']
    signal = hists['mC750_1l']
    data = hists['Data']
    
    plt.rcParams.update({'font.size': 14,'axes.titlesize': 18,'axes.labelsize': 18,
                         'xtick.labelsize': 12,'ytick.labelsize': 12})
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    hist.plot1d(lostLep, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillopts, overflow = 'over')
    hist.plot1d(otherbackgrounds, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillopts1, overflow = 'over')
    #hist.plot1d(signal, overlay="dataset", ax=ax, clear=False,density=False, stack=False, 
    #            error_opts=data_err_opts, overflow = 'over')
    hist.plot1d(data, overlay="dataset", ax=ax, clear=False,density=False, stack=False, 
                error_opts=data_err_opts_rat, overflow = 'over')
    ax.set_yscale('log')
    ax.set_ylim(0, 10000)
    ax.set_xlabel(None)
    leg = ax.legend()
    hist.plotratio(num=data.sum('dataset'), denom=background.sum('dataset'), ax=rax,
                   error_opts = data_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    rax.set_ylabel('Ratio')
    rax.set_ylim(0,2)
    fig.savefig(os.path.join(outdir, "{}_log.png".format(name)))
    fig.savefig(os.path.join(outdir, "{}_log.pdf".format(name)))
    fig.clear()

def savefigshape(hists, outdir, name):
    import re
    bkgandsig = re.compile('(?!Data)')
    bkganddata = re.compile('(?!mC750_l1)')
    allbutLL = re.compile('(?!LL)')
    
    background = hists[bkgandsig][bkganddata]
    otherbackgrounds = hists[bkgandsig][bkganddata][allbutLL]
    lostLep = hists['LL']
    signal = hists['mC750_1l']
    data = hists['Data']
    
    plt.rcParams.update({'font.size': 14,'axes.titlesize': 18,'axes.labelsize': 18,
                         'xtick.labelsize': 12,'ytick.labelsize': 12})
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    hist.plot1d(background, overlay="dataset",  ax=ax, clear=False, density=True, 
                stack=False,line_opts = lineopts2, overflow = 'over')
    #hist.plot1d(signal, overlay="dataset", ax=ax, clear = False, density=True, stack=False, 
    #            error_opts=data_err_opts, overflow = 'over')
    hist.plot1d(data, overlay="dataset", ax=ax, clear = False, density=True, stack=False, 
                error_opts=data_err_opts_rat, overflow = 'over')
    ax.set_yscale('log')
    ax.set_ylim(0, 10000)
    ax.set_xlabel(None)
    leg = ax.legend()
    hist.plotratio(num=data.sum('dataset'), denom=background.sum('dataset'), ax=rax,
                   error_opts = data_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    rax.set_ylabel('Ratio')
    rax.set_ylim(0,2)
    ax.figure.savefig(os.path.join(outdir, "{}_shape_log.png".format(name)))
    ax.figure.savefig(os.path.join(outdir, "{}_shape_log.pdf".format(name)))
    ax.clear()

In [9]:
#Let's remind ourselves of the histograms we created so we can loop through them 
#and create an array to loop through when we rebin. 
histograms = ["h_pt_met200_CR",
              "h_pt_met400_CR",
              "h_pt_met600_CR",
              "fj_pt_met200_CR",
              "fj_pt_met400_CR",
              "fj_pt_met600_CR",
              "h_mass_met200_CR",
              "h_mass_met400_CR",
              "h_mass_met600_CR",
              "met_noHiggs",
              "met_oneHiggs",
              "met_oneW",
              "met_inHiggs",
              "met_outHiggs",
             ]

overlays   = ["met_CR",
              "met_SR",
             ]

#Make sure this points to a directory you can print to!
outdir = "/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/tutorialPlots/"

In [10]:
for name in histograms:
    print (name)
    hists = output[name]

    if name == "h_pt_met200_CR":
        new_pt_bins = hist.Bin('pt', r'Lead Higgs pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "h_pt_met400_CR":
        new_pt_bins = hist.Bin('pt', r'Lead Higgs pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "h_pt_met600_CR":
        new_pt_bins = hist.Bin('pt', r'Lead Higgs pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)

        

    if name == "fj_pt_met200_CR":
        new_pt_bins = hist.Bin('pt', r'Lead FatJet pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "fj_pt_met400_CR":
        new_pt_bins = hist.Bin('pt', r'Lead FatJet pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "fj_pt_met600_CR":
        new_pt_bins = hist.Bin('pt', r'Lead FatJet pT', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
        


    if name == "h_mass_met200_CR":
        new_mass_bins = hist.Bin('mass', r'Lead Higgs mass', 15, 0, 300)
        hists = hists.rebin('mass', new_mass_bins)
    
    if name == "h_mass_met400_CR":
        new_mass_bins = hist.Bin('mass', r'Lead Higgs mass', 15, 0, 300)
        hists = hists.rebin('mass', new_mass_bins)
    
    if name == "h_mass_met600_CR":
        new_mass_bins = hist.Bin('mass', r'Lead Higgs mass', 15, 0, 300)
        hists = hists.rebin('mass', new_mass_bins)
    
    
    
    if name == "met_noHiggs":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "met_oneHiggs":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "met_oneW":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "met_inHiggs":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
        
    if name == "met_outHiggs":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)        
        
    savefig(hists, outdir, name)
    #savefigshape(hists, outdir, name)

h_pt_met200_CR


AssertionError: len along main axis of h has to be smaller by 1 than len of bins

In [11]:
data_err_opts = {
    'linestyle': 'none',
    'marker': '_',
    'markersize': 10.,
    'color': 'r',
    'elinewidth': 1}

ST_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color':'#1982C4',
    'elinewidth': 1}

ttbar_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color':'#F76F8E',
    'elinewidth': 1}

wjets_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color':'#8AC926',
    'elinewidth': 1}

ttW_err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color':'#FFCA3A',
    'elinewidth': 1}



lineOverlayOpts = {
    'color': [('#1982C4'),('#F76F8E'),('#8AC926'),('#FFCA3A')]#, ('#FF5714'), ('#6A0136') ],
}



fillOverlayOpts = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#1982C4')],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillOverlayOpts1 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#F76F8E')],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillOverlayOpts2 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#8AC926') ],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillOverlayOpts3 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#FFCA3A') ],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillOverlayOpts4 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#FF5714') ],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}

fillOverlayOpts5 = {
    'edgecolor': (0,0,0,0.3),
    'facecolor': [('#6A0136') ],#, ('#A33B20'), ('#680E4B'), ('#F6AE2D'),('#45503B')],
    #'facecolor': [('#1467cc'), ('#51d673') ,('#f7d969'), ('#af84f0'), ('#4f842e'), ('#1ff4ff'),('#3612ab')],
}


outdir = "/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/tutorialPlots/"

In [26]:
def saveoverlay(histsCR, histsSR, outdir, name):
    import re
    #bkgandsig = re.compile('(?!Data)')
    bkgonly = re.compile('(?!mC750_l1)')
    allbutLL = re.compile('(?!LL)')
    allbutZ = re.compile('(?!ZNuNu)')
    
    #background = hists[bkgonly]
    QCDCR = histsCR['QCD']
    QCDSR = histsSR['QCD']
    lostLepCR = histsCR['LL']
    lostLepSR = histsSR['LL']
    ZCR = histsCR['ZNuNu']
    ZSR = histsSR['ZNuNu']
    #signal = hists['mC750_1l']
    #data = hists['Data']
    
    plt.rcParams.update({'font.size': 14,'axes.titlesize': 18,'axes.labelsize': 18,
                         'xtick.labelsize': 12,'ytick.labelsize': 12})
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    hist.plot1d(lostLepCR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts, overflow = 'over')
    hist.plot1d(lostLepSR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts1, overflow = 'over')
    hist.plot1d(ZSR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts3, overflow = 'over')
    hist.plot1d(ZCR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts2, overflow = 'over')
    hist.plot1d(QCDSR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts5, overflow = 'over')
    hist.plot1d(QCDCR, overlay="dataset",  ax=ax, clear=False, density=False, stack=True,
                fill_opts = fillOverlayOpts4, overflow = 'over')
    #hist.plot1d(signal, overlay="dataset", ax=ax, clear=False,density=False, stack=False, 
    #            error_opts=data_err_opts, overflow = 'over')
    #hist.plot1d(data, overlay="dataset", ax=ax, clear=False,density=False, stack=False, 
    #            error_opts=data_err_opts_rat, overflow = 'over')
    ax.set_yscale('log')
    ax.set_ylim(0, 10000)
    ax.set_xlabel(None)
    leg = ax.legend()
    #hist.plotratio(num=data.sum('dataset'), denom=background.sum('dataset'), ax=rax,
    #               error_opts = data_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
    #               unc='num', overflow = 'over')
    rax.set_ylabel('Ratio')
    rax.set_ylim(0,2)
    fig.savefig(os.path.join(outdir, "{}_log.png".format(name)))
    fig.savefig(os.path.join(outdir, "{}_log.pdf".format(name)))
    fig.clear()
    
    
def saveoverlayshape(hists, outdir, name):
    import re
    #bkgandsig = re.compile('(?!Data)')
    bkgonly = re.compile('(?!mC750_l1)')
    allbutLL = re.compile('(?!LL)')
    allbutZ = re.compile('(?!ZNuNu)')
    
    background = hists[bkgonly]
    ttbar = hists['TTJets']
    ttbar_normed = ttbar/ttbar.sum('dataset')
    ST = hists['ST']
    wjets = hists['WJets']
    ttw = hists['ttW/ttZ']
    
    #signal = hists['mC750_1l']
    #data = histsCR['Data']
    
    plt.rcParams.update({'font.size': 14,'axes.titlesize': 18,'axes.labelsize': 18,
                         'xtick.labelsize': 12,'ytick.labelsize': 12})
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    hist.plot1d(background, overlay="dataset",  ax=ax, clear=False, density = True, stack=False,
                line_opts = lineOverlayOpts, overflow = 'over')
    ax.set_yscale('log')
    ax.set_ylim(0, 10000)
    ax.set_xlabel(None)
    leg = ax.legend()
    hist.plotratio(num=ST.sum('dataset'), denom=ttbar.sum('dataset'), ax=rax, clear = False,
                   error_opts = ST_err_opts_rat, denom_fill_opts={}, guide_opts={},  
                   unc='num', overflow = 'over')
    hist.plotratio(num=wjets.sum('dataset'), denom=ttbar.sum('dataset'), ax=rax, clear = False,
                   error_opts = wjets_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    hist.plotratio(num=ttw.sum('dataset'), denom=ttbar.sum('dataset'), ax=rax, clear = False,
                   error_opts = ttW_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    hist.plotratio(num=ttbar.sum('dataset'), denom=ttbar.sum('dataset'), ax=rax, clear = False,
                   error_opts = ttbar_err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    rax.set_ylabel('Ratio')
    rax.set_ylim(0,2)
    fig.savefig(os.path.join(outdir, "{}_log.png".format(name)))
    fig.savefig(os.path.join(outdir, "{}_log.pdf".format(name)))
    fig.clear()

In [27]:
histsCR = output["met_CR"]
histsSR = output["met_SR"]
#print(type(histsCR))
new_pt_bins = hist.Bin('pt',r'MET',20,200,1000)
histsCR = histsCR.rebin('pt',new_pt_bins)
histsSR = histsSR.rebin('pt',new_pt_bins)
#saveoverlay(histsCR,histsSR,outdir,"MET_overlay")
saveoverlayshape(histsCR,outdir,"MET_CR_overlay_shape")
saveoverlayshape(histsSR,outdir,"MET_SR_overlay_shape")

histsCR = output["met_Higgs_CR"]
histsSR = output["met_Higgs_SR"]
new_pt_bins = hist.Bin('pt',r'MET',20,200,1000)
histsCR = histsCR.rebin('pt',new_pt_bins)
histsSR = histsSR.rebin('pt',new_pt_bins)
#saveoverlay(histsCR,histsSR,outdir,"MET_Higgs_overlay")
saveoverlayshape(histsCR,outdir,"MET_CR_Higgs_overlay_shape")
saveoverlayshape(histsSR,outdir,"MET_SR_Higgs_overlay_shape")

histsCR = output["met_W_CR"]
histsSR = output["met_W_SR"]
new_pt_bins = hist.Bin('pt',r'MET',20,200,1000)
histsCR = histsCR.rebin('pt',new_pt_bins)
histsSR = histsSR.rebin('pt',new_pt_bins)
#saveoverlay(histsCR,histsSR,outdir,"MET_W_overlay")
saveoverlayshape(histsCR,outdir,"MET_CR_W_overlay_shape")
saveoverlayshape(histsSR,outdir,"MET_SR_W_overlay_shape")

histsCR = output["met_Higgs_W_CR"]
histsSR = output["met_Higgs_W_SR"]
new_pt_bins = hist.Bin('pt',r'MET',20,200,1000)
histsCR = histsCR.rebin('pt',new_pt_bins)
histsSR = histsSR.rebin('pt',new_pt_bins)
#saveoverlay(histsCR,histsSR,outdir,"MET_Higgs_W_overlay")
saveoverlayshape(histsCR,outdir,"MET_CR_Higgs_W_overlay_shape")
saveoverlayshape(histsSR,outdir,"MET_SR_Higgs_W_overlay_shape")

TypeError: unsupported operand type(s) for /: 'Hist' and 'Hist'

In [51]:
for name in overlays:
    print (name)
    hists = output[name]

    if name == "met_CR":
        new_pt_bins = hist.Bin('pt', r'MET', 20, 200, 1000)
        hists = hists.rebin('pt', new_pt_bins)
    
    if name == "met_SR":
           
        
    savefig(hists, outdir, name)
    #savefigshape(hists, outdir, name)

IndentationError: expected an indented block (<ipython-input-51-1bd7e780cfbe>, line 12)